In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

In [2]:
from thesis.config import load_config, GaussianSplattingSettings

path = "configs/single_frame.yml"
config = load_config(path)

# audio smoothing